In [26]:
from types import resolve_bases
import numpy as np
import torch
import pickle
import glob
import json
import time

from networks import Decoder
from dataLoader import DatasetDecoder
from marching_cubes_rgb import *

import IPython

# directory which contain the SDF input and where all the output will be generated
MAIN_DIR = "../../image2sdf/"
# MAIN_DIR = "/home/loic/MasterPDM/image2sdf/"

DECODER_PATH = "models_and_codes/decoderSDF.pth"
LATENT_CODE_PATH = "models_and_codes/latent_code.pkl"
PARAM_FILE = "config/param.json"

SDF_DIR = MAIN_DIR + "sdf/"
RESOLUTION = 64



def init_xyz(resolution):
    """ fill 3d grid representing 3d location to give as input to the decoder """
    xyz = torch.empty(resolution * resolution * resolution, 3).cuda()

    for x in range(resolution):
        for y in range(resolution):
            for z in range(resolution):
                xyz[x * resolution * resolution + y * resolution + z, :] = torch.Tensor([x/(resolution-1)-0.5,y/(resolution-1)-0.5,z/(resolution-1)-0.5])

    return xyz

def init_lat_vecs(num_scenes, latent_size):
    """initialize random latent code for every model"""

    lat_code_mu = torch.nn.Embedding(num_scenes, latent_size).cuda()
    torch.nn.init.normal_(
        lat_code_mu.weight.data,
        0.0,
        1.0,
    )
    lat_code_log_std = torch.nn.Embedding(num_scenes, latent_size).cuda()
    torch.nn.init.normal_(
        lat_code_log_std.weight.data,
        0.0,
        0.0,
    )

    return lat_code_mu, lat_code_log_std

def init_opt_sched(decoder, lat_vecs_mu, lat_vecs_log_std, param):
    """ initialize optimizer and scheduler"""

    optimizer = torch.optim.Adam(
        [
            {
                "params": decoder.parameters(),
                "lr": param["eta_decoder"],
            },
            {
                "params": lat_vecs_mu.parameters(),
                "lr": param["eta_latent_space_mu"],
            },
            {
                "params": lat_vecs_log_std.parameters(),
                "lr": param["eta_latent_space_std"],
            },
        ]
    )

    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=param["gammaLR"])

    return optimizer, scheduler

if __name__ == '__main__':
    print("Loading parameters...")

    # load parameters
    param_all = json.load(open(PARAM_FILE))
    param = param_all["decoder"]

    resolution = RESOLUTION
    num_samples_per_model = resolution * resolution * resolution
    threshold_precision = 1.0/resolution

    # get models' hashs
    list_model_hash = []
    for val in glob.glob(SDF_DIR + "*.h5"):
        list_model_hash.append(os.path.basename(val).split('.')[0])
    num_model = len(list_model_hash)

    # dataLoader for training dataset
    training_dataset = DatasetDecoder(list_model_hash, SDF_DIR, resolution)
    training_generator = torch.utils.data.DataLoader(training_dataset, **param["dataLoader"])

    # fill a xyz grid to give as input to the decoder 
    xyz = init_xyz(resolution)
    # initialize a random latent code for each models
    lat_code_mu, lat_code_log_std = init_lat_vecs(num_model, param["latent_size"])

    idx = torch.arange(num_model).type(torch.LongTensor).cuda()
    dict_model_hash_2_idx = dict()
    for model_hash, i in zip(list_model_hash, range(num_model)):
        dict_model_hash_2_idx[model_hash] = idx[i]

    # decoder
    decoder = Decoder(param["latent_size"]).cuda()
    loss = torch.nn.MSELoss(reduction='none')
    # initialize optimizer and scheduler
    optimizer, scheduler = init_opt_sched(decoder, lat_code_mu, lat_code_log_std, param["optimizer"])




    print("Start training...")

    for hash, sdf_gt, rgb_gt in training_generator:
        # only 1 sample per batch!
        hash = hash[0]
        sdf_gt = sdf_gt[0]
        rgb_gt = rgb_gt[0]

        sdf_gt = sdf_gt.cuda()
        rgb_gt = rgb_gt.cuda()

        code_mu, code_log_std = lat_code_mu(dict_model_hash_2_idx[hash]), lat_code_log_std(dict_model_hash_2_idx[hash])
        latent_code =  torch.empty(num_samples_per_model, param["latent_size"]).normal_().cuda() * code_log_std.exp() * param["lambda_variance"] + code_mu


        pred = decoder(latent_code, xyz)


 

Using skimage's marching cubes implementation.
Loading parameters...
Start training...
37126540









RuntimeError: The size of tensor a (262144) must match the size of tensor b (6) at non-singleton dimension 0

In [47]:
a = torch.empty(num_samples_per_model, param["latent_size"]).normal_().cuda()

In [54]:
code_log_std[1]

tensor(0.1000, device='cuda:0', grad_fn=<SelectBackward>)

In [55]:
code_log_std.exp()

tensor([1.0000, 1.1052, 1.0000, 1.0000, 1.0000, 1.0000], device='cuda:0',
       grad_fn=<ExpBackward>)

In [56]:
a * code_log_std.exp()

tensor([[-1.6213e+00, -4.9051e-01,  9.8526e-01, -1.7783e+00,  8.7508e-01,
         -5.7617e-01],
        [ 6.9454e-01, -7.1401e-01,  3.0229e+00,  1.6118e-01, -2.0640e+00,
          1.3472e-03],
        [-5.6624e-01, -6.3003e-01, -4.4470e-01,  1.5505e+00, -5.0576e-01,
         -1.8991e+00],
        ...,
        [ 4.1694e-01, -2.2405e+00,  1.5258e-01, -1.0663e+00, -2.0300e+00,
         -8.9969e-01],
        [ 2.0419e-01,  8.9827e-01,  9.4146e-01, -1.9243e-01, -5.2128e-01,
          1.5413e+00],
        [-7.8587e-03,  2.0081e-02, -2.0799e-02,  9.9405e-01, -8.5101e-01,
         -3.2946e-02]], device='cuda:0', grad_fn=<MulBackward0>)

In [50]:
a

tensor([[-1.6213e+00, -4.4383e-01,  9.8526e-01, -1.7783e+00,  8.7508e-01,
         -5.7617e-01],
        [ 6.9454e-01, -6.4606e-01,  3.0229e+00,  1.6118e-01, -2.0640e+00,
          1.3472e-03],
        [-5.6624e-01, -5.7007e-01, -4.4470e-01,  1.5505e+00, -5.0576e-01,
         -1.8991e+00],
        ...,
        [ 4.1694e-01, -2.0273e+00,  1.5258e-01, -1.0663e+00, -2.0300e+00,
         -8.9969e-01],
        [ 2.0419e-01,  8.1279e-01,  9.4146e-01, -1.9243e-01, -5.2128e-01,
          1.5413e+00],
        [-7.8587e-03,  1.8170e-02, -2.0799e-02,  9.9405e-01, -8.5101e-01,
         -3.2946e-02]], device='cuda:0')

In [43]:
code_log_std

tensor([0., 0., 0., 0., 0., 0.], device='cuda:0', grad_fn=<EmbeddingBackward>)

In [39]:
b = torch.tensor([[1,2],[3,4],[1,1],[1,1]])

In [41]:
a* b 

tensor([[1, 4],
        [3, 8],
        [1, 2],
        [1, 2]])